In [18]:
import pandas as pd
import numpy as np
import geopandas as gpd

## Load parks

In [3]:
df_parks = gpd.read_file(
    "https://data.cityofnewyork.us/resource/enfh-gkve.geojson?$limit=1000000"
)

In [4]:
df_parks[df_parks["communityboard"] == "103"].to_file("data/parks.geojson")

## Load subsidized housing

In [2]:
# df_subsidized = pd.read_csv("https://furmancenter.org/files/CoreData/FC_SHD_bbl_analysis_2024-08-29.csv")
df_subsidized = pd.read_csv("data/FC_SHD_bbl_analysis_2024-08-29.csv")
df_subsidized.columns[30:]

/var/folders/_q/g5l2s5950hz1zr9syt43gzp80000gn/T/ipykernel_74985/3565231597.py:2: DtypeWarning: Columns (103,106,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_subsidized = pd.read_csv("data/FC_SHD_bbl_analysis_2024-08-29.csv")


Index(['data_dof', 'data_hcrlihtc', 'data_hpd', 'data_hudcon', 'data_hudfin',
       'data_hudlihtc', 'data_ml', 'data_nycha', 'sub_fin', 'sub_prog',
       'sub_tax', 'sub_zone', 'prog_202_8', 'start_202_8', 'end_202_8',
       'prog_221d', 'start_221d', 'end_221d', 'prog_223', 'start_223',
       'end_223', 'prog_420c', 'start_420c', 'end_420c', 'prog_421a',
       'start_421a', 'end_421a', 'prog_421a_aff', 'start_421a_aff',
       'end_421a_aff', 'prog_8a', 'start_8a', 'end_8a', 'prog_hpd_oth',
       'start_hpd_oth', 'end_hpd_oth', 'prog_hud_fin_oth', 'start_hud_fin_oth',
       'end_hud_fin_oth', 'prog_hud_proj_oth', 'start_hud_proj_oth',
       'end_hud_proj_oth', 'prog_inclus_hous', 'start_inclus_hous',
       'end_inclus_hous', 'prog_j51', 'start_j51', 'end_j51', 'prog_lamp',
       'start_lamp', 'end_lamp', 'prog_lihtc4', 'start_lihtc4', 'end_lihtc4',
       'prog_lihtc9', 'start_lihtc9', 'end_lihtc9', 'prog_lmsa', 'start_lmsa',
       'end_lmsa', 'prog_mfp', 'start_mfp', 'end

In [3]:
# load mappluto
# pluto = gpd.read_file("data/nyc_mappluto_25v1_1_fgdb/MapPLUTO25v1_1.gdb")
# plutocb3 = pluto[pluto['CD']==103]
# plutocb3.to_file('data/cb3_pluto.geojson')
plutocb3 = gpd.read_file("data/cb3_pluto.geojson")

In [4]:
plutocb3_subhousing = plutocb3.merge(
    df_subsidized.rename(columns={"bbl": "BBL"}), on="BBL"
)

In [5]:
nycha = plutocb3_subhousing[
    (plutocb3_subhousing["prog_nycha_mix"] == 1)
    | (plutocb3_subhousing["prog_nycha_ph"] == 1)
]
nycha["Address"] = nycha["Address"].str.strip()

/opt/anaconda3/envs/cb3-census/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
# load nycha development names
nycha_names = gpd.read_file(
    "https://data.cityofnewyork.us/resource/3ub5-4ph8.geojson?$limit=100000"
)
nycha_names = nycha_names[
    (nycha_names["community_district"] == "3") & (nycha_names["borough"] == "MANHATTAN")
]
nycha_manual = pd.read_excel("data/nycha_development_names.xlsx")
nycha_names = pd.concat(
    [nycha_names[["address", "development"]], nycha_manual], axis=0
).drop_duplicates()
nycha_names["address"] = nycha_names["address"].str.strip()
nycha_mgd = nycha.merge(
    nycha_names[["address", "development"]].drop_duplicates(),
    right_on="address",
    left_on="Address",
    how="left",
)
nycha_mgd[["Address", "development", "geometry", "OwnerName"]].to_crs(4326).to_file(
    "data/nycha.geojson"
)

In [7]:
sec8 = plutocb3_subhousing[
    (plutocb3_subhousing["prog_proj8"] == 1) | (plutocb3_subhousing["prog_rad"] == 1)
]
sec8.to_crs(4326).to_file("data/sec8.geojson")

In [24]:
ml = plutocb3_subhousing[(plutocb3_subhousing["prog_ml"] == 1)]


In [25]:
ml['name'] = np.select([ml['Address'] == '100 CHERRY STREET', ml['Address'] == '645 WATER STREET', 
ml['Address'] == '605 WATER STREET', ml['Address'] == '604 WATER STREET', 
ml['Address'] == '35 MONTGOMERY STREET', ml['Address'] == '305 MADISON STREET', 
ml['Address'] == '40 1 AVENUE', ml['Address'] == '60 1 AVENUE', ml['Address'] == '66 1 AVENUE', 
ml['Address'] == '87 DELANCEY STREET', ml['Address'] == '162 AVENUE C', 
ml['Address'] == '620 EAST 13 STREET'], 
['Knickerbocker Village', 'Gouveneur Gardens', 'Gouveneur Gardens', 'Gouveneur Gardens', 
'Gouveneur Gardens', 'Gouveneur Gardens', 'Village View', 'Village View', 'Village View', 
'Masaryk Towers', 'Village East', 'Tanya Towers'], '')
ml.to_crs(4326).to_file("data/ml.geojson")

/opt/anaconda3/envs/cb3-census/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [26]:
knickerbocker_village = plutocb3[plutocb3["BBL"].astype(str).str.contains("002530001")][
    ["Address", "OwnerName", "geometry"]
]
knickerbocker_village.to_crs(4326).to_file("data/knickerbocker_village.geojson")

## Load Crashes

In [219]:
# load 2023 sectors
gdf_sectors = gpd.read_file("data/acs_2023_mapbox.geojson")

In [209]:
gdf_crash_2023 = gpd.read_file(
    "https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$limit=10000000&$where=date_extract_y(crash_date)=2023"
)
gdf_crash_2024 = gpd.read_file(
    "https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$limit=10000000&$where=date_extract_y(crash_date)=2024"
)
gdf_crash_2025 = gpd.read_file(
    "https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$limit=10000000&$where=date_extract_y(crash_date)=2025"
)

Skipping field crash_time: unsupported OGR type: 10
Skipping field crash_time: unsupported OGR type: 10
Skipping field crash_time: unsupported OGR type: 10


In [220]:
gdf_crash = pd.concat([gdf_crash_2025, gdf_crash_2024, gdf_crash_2023])

In [221]:
# spatial join, add data columns
columns_to_drop = [
    x for x in gdf_sectors.columns if x in gdf_crash.columns and x != "geometry"
]
gdf_crash_cb3 = gdf_crash.sjoin(gdf_sectors.drop(columns_to_drop, axis=1), how="inner")
gdf_crash_cb3["crash_date"] = pd.to_datetime(gdf_crash_cb3["crash_date"])
gdf_crash_cb3["crash_month"] = gdf_crash_cb3["crash_date"].dt.month
gdf_crash_cb3["crash_year"] = gdf_crash_cb3["crash_date"].dt.year
gdf_crash_cb3.head()

,location_state,number_of_motorist_injured,zip_code,number_of_cyclist_killed,location_zip,contributing_factor_vehicle_3,latitude,on_street_name,collision_id,number_of_persons_killed,...,white_diff,white_pct_change,white_pct_diff,white_pct_pct_change,with_income_diff,with_income_pct_change,with_income_pct_diff,with_income_pct_pct_change,crash_month,crash_year
36,,0,10009,0,,None,40.730442,E 14 ST,4790150,0,...,-1037.0,-0.079714,-0.061737,-0.081375,329.0,0.023068,0.030792,0.035387,2,2025
321,,1,10002,0,,None,40.713253,FRANKLIN D ROOSEVELT DR,4788959,0,...,108.0,0.015186,-0.066533,-0.106451,1358.0,0.151259,-0.003800,-0.004247,1,2025
351,,0,10002,0,,None,40.713554,MADISON ST,4793933,0,...,-132.0,-0.029101,0.000651,0.002916,266.0,0.018433,0.046660,0.056551,2,2025
508,,0,None,0,,None,40.708763,FDR DRIVE,4790508,0,...,-220.0,-0.102136,-0.007543,-0.055790,-984.0,-0.083256,-0.024846,-0.029717,1,2025
541,,1,10002,0,,None,40.71376,GRAND ST,4783783,0,...,-132.0,-0.029101,0.000651,0.002916,266.0,0.018433,0.046660,0.056551,1,2025


In [222]:
collision_cols = [
    "number_of_motorist_injured",
    "number_of_cyclist_killed",
    "number_of_cyclist_injured",
    "number_of_persons_killed",
    "number_of_persons_injured",
    "number_of_pedestrians_killed",
    "number_of_pedestrians_injured",
]
gdf_crash_cb3[collision_cols] = gdf_crash_cb3[collision_cols].astype(int)

In [223]:
# count number of collisions
collision_count = (
    gdf_crash_cb3[["sectors", "crash_month", "crash_year"]]
    .value_counts()
    .rename("collision_count")
    .reset_index()
)
collision_count.head()

,sectors,crash_month,crash_year,collision_count
0,9,3,2023,29
1,9,10,2023,29
2,9,11,2023,28
3,9,5,2024,27
4,9,3,2024,26


In [224]:
gdf_crash_cb3_total = (
    gdf_crash_cb3.groupby(["sectors", "crash_month", "crash_year"], as_index=False)[
        collision_cols
    ]
    .sum()
    .fillna(0)
    .merge(collision_count, on=["sectors", "crash_month", "crash_year"], how="outer")
)
collision_cols = collision_cols + ["collision_count"]
gdf_crash_cb3_total

,sectors,crash_month,crash_year,number_of_motorist_injured,number_of_cyclist_killed,number_of_cyclist_injured,number_of_persons_killed,number_of_persons_injured,number_of_pedestrians_killed,number_of_pedestrians_injured,collision_count
0,1,1,2023,1,0,3,0,4,0,0,12
1,1,1,2024,2,0,2,0,4,0,0,11
2,1,1,2025,1,0,1,0,4,0,2,11
3,1,2,2023,2,0,3,0,5,0,0,6
4,1,2,2024,9,0,0,0,9,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...
284,9,10,2024,5,0,3,0,14,0,6,22
285,9,11,2023,8,0,4,0,15,0,3,28
286,9,11,2024,2,0,1,0,7,0,4,12
287,9,12,2023,3,0,4,0,8,0,1,21


In [225]:
full_index = pd.MultiIndex.from_product(
    [
        gdf_crash_cb3_total["sectors"].unique(),
        gdf_crash_cb3_total["crash_month"].unique(),
        gdf_crash_cb3_total["crash_year"].unique(),
    ],
    names=["sectors", "crash_month", "crash_year"],
)

In [226]:
gdf_crash_cb3_complete = (
    gdf_crash_cb3_total.set_index(["sectors", "crash_month", "crash_year"])
    .reindex(full_index)
    .reset_index()
)
gdf_crash_cb3_complete = gdf_crash_cb3_complete[
    ~(
        (gdf_crash_cb3_complete["crash_month"] > 5)
        & (gdf_crash_cb3_complete["crash_year"] == 2025)
    )
]
gdf_crash_cb3_complete[collision_cols] = gdf_crash_cb3_complete[collision_cols].fillna(
    0
)
gdf_crash_cb3_complete.shape, gdf_crash_cb3_total.shape

((290, 11), (289, 11))

In [227]:
gdf_crash_cb3_summary = gdf_crash_cb3_complete.groupby("sectors")[collision_cols].mean()
gdf_crash_cb3_summary

,number_of_motorist_injured,number_of_cyclist_killed,number_of_cyclist_injured,number_of_persons_killed,number_of_persons_injured,number_of_pedestrians_killed,number_of_pedestrians_injured,collision_count
sectors,,,,,,,,
1,3.827586,0.0,3.206897,0.034483,9.310345,0.034483,1.689655,14.689655
10,2.931034,0.0,0.103448,0.068966,3.310345,0.068966,0.172414,7.241379
2,1.241379,0.0,1.655172,0.000000,4.275862,0.000000,1.172414,7.310345
3,1.241379,0.0,0.724138,0.103448,2.827586,0.068966,0.551724,5.827586
4,2.379310,0.0,1.206897,0.000000,4.827586,0.000000,0.896552,9.965517
5,3.586207,0.0,0.793103,0.034483,5.724138,0.034483,1.206897,9.827586
6,2.379310,0.0,0.551724,0.137931,4.620690,0.137931,1.379310,9.689655
7,2.310345,0.0,0.517241,0.000000,4.068966,0.000000,1.068966,8.379310
8,1.827586,0.0,1.551724,0.068966,5.241379,0.068966,1.689655,11.689655


In [228]:
gdf_sectors.drop(columns_to_drop, axis=1).merge(
    gdf_crash_cb3_summary, on="sectors"
).to_file("data/acs_2023_mapbox.geojson")